<a href="https://colab.research.google.com/github/Marcin19721205/IntroductionToBigDataWSB/blob/main/Zadanie_Nr12_MJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import kagglehub

# Download latest version
path = kagglehub.dataset_download("iabhishekofficial/mobile-price-classification")

print("Path to dataset files:", path)

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("iabhishekofficial/mobile-price-classification")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'mobile-price-classification' dataset.
Path to dataset files: /kaggle/input/mobile-price-classification


In [4]:
#stack
#%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import pandas as pd
import plotly.express as px
import tensorflow as tf
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout

tf.__version__

'2.19.0'

In [6]:
#dataset MobilePhonePricePrediction

import pandas as pd

# Wczytanie plików z katalogu sample_data
#data - cały plik danych, data & labels
dataraw = pd.read_csv("sample_data/train.csv")

print("=== data — head ===")
print(dataraw.head(), "\n")

print("=== data — info ===")
print(dataraw.info(), "\n")

print("=== data — describe ===")
print(dataraw.describe(), "\n")

print("=== data — datashape ===")
print(dataraw.shape)


=== data — head ===
   battery_power  blue  clock_speed  dual_sim  fc  four_g  int_memory  m_dep  \
0            842     0          2.2         0   1       0           7    0.6   
1           1021     1          0.5         1   0       1          53    0.7   
2            563     1          0.5         1   2       1          41    0.9   
3            615     1          2.5         0   0       0          10    0.8   
4           1821     1          1.2         0  13       1          44    0.6   

   mobile_wt  n_cores  ...  px_height  px_width   ram  sc_h  sc_w  talk_time  \
0        188        2  ...         20       756  2549     9     7         19   
1        136        3  ...        905      1988  2631    17     3          7   
2        145        5  ...       1263      1716  2603    11     2          9   
3        131        6  ...       1216      1786  2769    16     8         11   
4        141        2  ...       1208      1212  1411     8     2         15   

   three_g  touch_

Podział na dane i labele
na kopii danych

In [7]:
#kopia
data = dataraw.copy()

#wydzielenie labels
labels = data.pop("price_range")    # kolumna docelowa (y)

# --- WYDZIELENIE CECH ---
features = data                     # pozostałe kolumny (X)

print("=== FEATURES (X) ===")
print(features.head(), "\n")

print("=== LABELS (y) ===")
print(labels.head())


=== FEATURES (X) ===
   battery_power  blue  clock_speed  dual_sim  fc  four_g  int_memory  m_dep  \
0            842     0          2.2         0   1       0           7    0.6   
1           1021     1          0.5         1   0       1          53    0.7   
2            563     1          0.5         1   2       1          41    0.9   
3            615     1          2.5         0   0       0          10    0.8   
4           1821     1          1.2         0  13       1          44    0.6   

   mobile_wt  n_cores  pc  px_height  px_width   ram  sc_h  sc_w  talk_time  \
0        188        2   2         20       756  2549     9     7         19   
1        136        3   6        905      1988  2631    17     3          7   
2        145        5   6       1263      1716  2603    11     2          9   
3        131        6   9       1216      1786  2769    16     8         11   
4        141        2  14       1208      1212  1411     8     2         15   

   three_g  touch_scree

#Normalizacja

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# features = X (dane)
# labels   = y (price_range)

# --- 70% train, 15% val, 15% test ---
X_trainval, X_test, y_trainval, y_test = train_test_split(
    features, labels,
    test_size=0.15,
    random_state=42,
    stratify=labels
)

X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval,
    test_size=0.176,
    random_state=42,
    stratify=y_trainval
)

# --- Normalizacja ---
scaler = StandardScaler()
scaler.fit(X_train)  # uczymy scaler TYLKO na treningowym

X_train_scaled = scaler.transform(X_train)
X_val_scaled   = scaler.transform(X_val)
X_test_scaled  = scaler.transform(X_test)

print("X_train_scaled shape:", X_train_scaled.shape)
print("X_val_scaled shape:",   X_val_scaled.shape)
print("X_test_scaled shape:",  X_test_scaled.shape)


X_train_scaled shape: (1400, 20)
X_val_scaled shape: (300, 20)
X_test_scaled shape: (300, 20)


In [9]:
print(f'X_train_scalled shape: {X_train.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'X_test_scaled shape: {X_test.shape}')
print(f'y_test shape: {y_test.shape}')

X_train_scalled shape: (1400, 20)
y_train shape: (1400,)
X_test_scaled shape: (300, 20)
y_test shape: (300,)


#Model Sieci

In [10]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(20,))) #warstwa wejściowa - shape danych 20 - 20 kolumn
model.add(Dense(units=256, activation='relu')) #pierwsza ukryta
model.add(Dropout(0.2))
model.add(Dense(units=128, activation='relu')) #druga ukryta
model.add(Dropout(0.2))
model.add(Dense(units=4, activation='softmax')) #wyjściowa

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │         8,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 42,532 (166.14 KB)

 Trainable params: 42,532 (166.14 KB)

 Non-trainable params: 0 (0.00 B)

Model train

In [11]:
history=model.fit(X_train_scaled, y_train, epochs=50, validation_split=0.2, batch_size=32)

Epoch 1/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.3863 - loss: 1.3140 - val_accuracy: 0.5464 - val_loss: 1.0466
Epoch 2/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6378 - loss: 0.9100 - val_accuracy: 0.6929 - val_loss: 0.6896
Epoch 3/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7335 - loss: 0.6315 - val_accuracy: 0.8036 - val_loss: 0.5157
Epoch 4/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7958 - loss: 0.4880 - val_accuracy: 0.8250 - val_loss: 0.4445
Epoch 5/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8459 - loss: 0.3777 - val_accuracy: 0.8393 - val_loss: 0.3886
Epoch 6/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8562 - loss: 0.3401 - val_accuracy: 0.8536 - val_loss: 0.3315
Epoch 7/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8836 - loss: 0.2873 - val_accuracy: 0.8821 - val_loss: 0.3061
Epoch 8/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9045 - loss: 0.2388 - val_accuracy: 0.8821 - val_loss

In [12]:
model.layers #wyświetl warstwy

[<Dense name=dense, built=True>,
 <Dense name=dense_1, built=True>,
 <Dropout name=dropout, built=True>,
 <Dense name=dense_2, built=True>,
 <Dropout name=dropout_1, built=True>,
 <Dense name=dense_3, built=True>]

In [13]:
# lista tensorów wejściowych do modelu
model.inputs

[<KerasTensor shape=(None, 20), dtype=float32, sparse=False, ragged=False, name=keras_tensor>]

In [14]:
# lista tensorów wyjściowych z modelu
model.outputs

[<KerasTensor shape=(None, 4), dtype=float32, sparse=False, ragged=False, name=keras_tensor_21>]

Historia

In [15]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.head()

,accuracy,loss,val_accuracy,val_loss,epoch
0,0.458929,1.228294,0.546429,1.046619,0
1,0.657143,0.829157,0.692857,0.689641,1
2,0.753571,0.595121,0.803571,0.515662,2
3,0.804464,0.467985,0.825000,0.444478,3
4,0.850893,0.372473,0.839286,0.388588,4


Metryki

In [16]:
metrics = pd.DataFrame(history.history)
metrics['epoch'] = history.epoch
metrics

,accuracy,loss,val_accuracy,val_loss,epoch
0,0.458929,1.228294,0.546429,1.046619,0
1,0.657143,0.829157,0.692857,0.689641,1
2,0.753571,0.595121,0.803571,0.515662,2
3,0.804464,0.467985,0.825000,0.444478,3
4,0.850893,0.372473,0.839286,0.388588,4
5,0.863393,0.325883,0.853571,0.331525,5
6,0.883929,0.290542,0.882143,0.306099,6
7,0.902679,0.239318,0.882143,0.286786,7
8,0.920536,0.204628,0.885714,0.267260,8
9,0.933929,0.177446,0.882143,0.262627,9


In [17]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['accuracy'], name='accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['loss'], name='loss'), row=1, col=2)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['val_accuracy'], name='val_accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['val_loss'], name='val_loss'), row=1, col=2)

fig.update_xaxes(title_text='epochs')
fig.update_yaxes(title_text='accuracy')
fig.update_layout(width=1000, title='Accuracy and Loss')
fig.show()

In [18]:
predictions = model.predict(X_test_scaled)
predictions

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


array([[0.0000000e+00, 2.1511532e-27, 3.7298881e-10, 9.9999994e-01],
       [9.9999994e-01, 5.8003209e-08, 3.5485234e-25, 0.0000000e+00],
       [9.9999517e-01, 4.7492290e-06, 1.7866036e-17, 3.9984171e-28],
       ...,
       [7.6582735e-24, 1.1614570e-11, 9.1857773e-01, 8.1422269e-02],
       [1.3657336e-05, 9.9992955e-01, 5.6753222e-05, 6.7876645e-13],
       [1.2119559e-14, 1.7829306e-04, 9.9982172e-01, 1.0061037e-08]],
      dtype=float32)

In [19]:
#prykcja jednoznaczna z przyporządkowaniem klas
pred_classes = predictions.argmax(axis=1)

# Tworzymy macierz one-hot (same 0/1)
pred_one_hot = np.zeros_like(predictions)
pred_one_hot[np.arange(len(pred_classes)), pred_classes] = 1

pred_one_hot[:10]


array([[0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.]], dtype=float32)

Funkcja sprawdzająca

In [23]:
import numpy as np
import pandas as pd

def check_predictions(predictions, y_test, n_classes=4, n_preview=10):
    #Predykcje
    pred_classes = predictions.argmax(axis=1)          # [2, 1, 0, 3, ...]

    #one-hot (bool)
    pred_one_hot = np.zeros_like(predictions, dtype=int)
    pred_one_hot[np.arange(len(pred_classes)), pred_classes] = 1

    #Prawdziwe etykiety -> one-hot
    y_true = np.array(y_test)                          # na wszelki wypadek
    true_one_hot = np.eye(n_classes, dtype=int)[y_true]

    #Błędy (czy cała próbka trafiona / nietrafiona)
    sample_correct = (pred_classes == y_true)
    n_errors = (~sample_correct).sum()
    n_total  = len(y_true)

    #Błędy bitowe (łączna liczba różnic 0/1 w one-hot)
    bit_errors = np.abs(true_one_hot - pred_one_hot).sum()

    #Podgląd kilku pierwszych wyników
    df_preview = pd.DataFrame({
        'y_true': y_true[:n_preview],
        'y_pred': pred_classes[:n_preview],
        'correct': sample_correct[:n_preview]
    })

    print("=== Podgląd pierwszych predykcji ===")
    print(df_preview, "\n")

    print("=== Przykładowe one-hot (prawda vs predykcja) ===")
    for i in range(min(n_preview, 5)):
        print(f"i={i}: true={true_one_hot[i]}, pred={pred_one_hot[i]}")
    print()

    print(f"Liczba próbek: {n_total}")
    print(f"Liczba błędnych próbek: {n_errors}")
    print(f"Accuracy: {(n_total - n_errors) / n_total:.4f}")
    print(f"Suma błędów bitowych (różnic 0/1 w one-hot): {bit_errors}")

    return {
        "pred_classes": pred_classes,
        "pred_one_hot": pred_one_hot,
        "true_one_hot": true_one_hot
    }
predictions = model.predict(X_test_scaled)

check_predictions(predictions, y_test)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
=== Podgląd pierwszych predykcji ===
   y_true  y_pred  correct
0       3       3     True
1       0       0     True
2       0       0     True
3       2       3    False
4       1       0    False
5       3       3     True
6       1       1     True
7       3       3     True
8       1       2    False
9       0       0     True 

=== Przykładowe one-hot (prawda vs predykcja) ===
i=0: true=[0 0 0 1], pred=[0 0 0 1]
i=1: true=[1 0 0 0], pred=[1 0 0 0]
i=2: true=[1 0 0 0], pred=[1 0 0 0]
i=3: true=[0 0 1 0], pred=[0 0 0 1]
i=4: true=[0 1 0 0], pred=[1 0 0 0]

Liczba próbek: 300
Liczba błędnych próbek: 38
Accuracy: 0.8733
Suma błędów bitowych (różnic 0/1 w one-hot): 76


{'pred_classes': array([3, 0, 0, 3, 0, 3, 1, 3, 2, 0, 0, 1, 1, 1, 1, 2, 3, 1, 1, 1, 0, 0,
        2, 0, 2, 0, 3, 1, 3, 3, 1, 2, 0, 1, 0, 0, 2, 1, 0, 0, 1, 1, 2, 3,
        0, 2, 3, 3, 0, 2, 2, 2, 3, 1, 3, 0, 1, 0, 3, 3, 0, 3, 2, 0, 1, 3,
        1, 0, 1, 0, 1, 3, 3, 3, 1, 2, 3, 0, 1, 1, 2, 2, 3, 2, 2, 0, 2, 2,
        1, 2, 1, 0, 1, 0, 0, 1, 3, 0, 3, 0, 2, 2, 0, 0, 2, 3, 1, 1, 3, 2,
        0, 2, 2, 1, 3, 1, 2, 0, 3, 2, 2, 1, 3, 3, 2, 1, 1, 3, 1, 0, 2, 3,
        2, 2, 0, 0, 1, 2, 2, 3, 0, 3, 0, 1, 2, 1, 3, 1, 3, 3, 3, 2, 0, 1,
        0, 1, 0, 2, 2, 2, 1, 0, 0, 3, 0, 2, 0, 3, 2, 1, 0, 2, 2, 1, 3, 3,
        0, 0, 2, 1, 1, 2, 3, 2, 2, 2, 0, 3, 1, 1, 3, 3, 2, 1, 1, 3, 1, 2,
        1, 3, 1, 1, 2, 1, 3, 0, 0, 2, 2, 1, 2, 2, 3, 1, 3, 1, 2, 0, 3, 1,
        2, 3, 0, 0, 3, 0, 0, 0, 1, 3, 1, 0, 0, 3, 3, 2, 3, 2, 0, 3, 0, 3,
        3, 1, 2, 2, 0, 3, 0, 0, 0, 0, 0, 2, 3, 2, 3, 2, 1, 3, 2, 1, 0, 0,
        1, 3, 2, 3, 3, 3, 3, 2, 0, 0, 3, 1, 2, 1, 2, 1, 3, 0, 0, 1, 0, 3,
        2, 0, 2, 1, 2,

In [21]:
test_loss, test_acc = model.evaluate(X_test_scaled, y_test, verbose=1)
print(test_acc)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8504 - loss: 0.7315 
0.8733333349227905


In [27]:
import pandas as pd
from sklearn.metrics import confusion_matrix

y_pred = predictions.argmax(axis=1)

cm = confusion_matrix(y_test, y_pred)

cm_df = pd.DataFrame(
    cm,
    index=[f"true_{i}" for i in range(4)],
    columns=[f"pred_{i}" for i in range(4)]
)

print("=== CONFUSION MATRIX (tabela) ===")
print(cm_df)


=== CONFUSION MATRIX (tabela) ===
        pred_0  pred_1  pred_2  pred_3
true_0      70       5       0       0
true_1       6      62       7       0
true_2       0       6      63       6
true_3       0       0       8      67


In [29]:
import plotly.express as px

fig = px.imshow(
    cm,
    text_auto=True,
    labels=dict(x="Predykcja", y="Rzeczywistość", color="Liczba"),
    x=[f"pred_{i}" for i in range(4)],
    y=[f"true_{i}" for i in range(4)],
    color_continuous_scale="Blues",
    title="Macierz konfuzji siec nuronowa — model klasyfikacji"
)

fig.update_layout(width=700, height=600)
fig.show()


#SVC NA TYCH SAMYCH DANYCH

In [30]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# --- 1. Klasyfikator SVC ---
svc_clf = SVC(
    C=1.0,
    kernel='rbf',
    gamma='scale',   # domyślne, zwykle ok na start
    decision_function_shape='ovr'   # one-vs-rest dla wielu klas
)

# --- 2. Trening na danych znormalizowanych ---
svc_clf.fit(X_train_scaled, y_train)

# --- 3. Accuracy na train / val / test ---
y_train_pred = svc_clf.predict(X_train_scaled)
y_val_pred   = svc_clf.predict(X_val_scaled)
y_test_pred  = svc_clf.predict(X_test_scaled)

print("Train accuracy:", accuracy_score(y_train, y_train_pred))
print("Val accuracy:  ", accuracy_score(y_val,   y_val_pred))
print("Test accuracy: ", accuracy_score(y_test,  y_test_pred))


Train accuracy: 0.9835714285714285
Val accuracy:   0.8733333333333333
Test accuracy:  0.89


In [32]:
import numpy as np
import pandas as pd

def check_predictions_svc(pred_classes, y_test, n_classes=4, n_preview=10):

    y_true = np.array(y_test)

    # one-hot prawda
    true_one_hot = np.eye(n_classes, dtype=int)[y_true]

    # one-hot predykcja
    pred_one_hot = np.eye(n_classes, dtype=int)[pred_classes]

    # poprawność
    sample_correct = (pred_classes == y_true)
    n_errors = (~sample_correct).sum()
    n_total  = len(y_true)

    # błędy bitowe
    bit_errors = np.abs(true_one_hot - pred_one_hot).sum()

    # podgląd
    df_preview = pd.DataFrame({
        'y_true': y_true[:n_preview],
        'y_pred': pred_classes[:n_preview],
        'correct': sample_correct[:n_preview]
    })

    print("=== Podgląd pierwszych predykcji (SVC) ===")
    print(df_preview, "\n")

    print("=== Przykładowe one-hot (prawda vs predykcja) ===")
    for i in range(min(n_preview, 5)):
        print(f"i={i}: true={true_one_hot[i]}, pred={pred_one_hot[i]}")
    print()

    print(f"Liczba próbek: {n_total}")
    print(f"Liczba błędnych próbek: {n_errors}")
    print(f"Accuracy: {(n_total - n_errors) / n_total:.4f}")
    print(f"Suma błędów bitowych: {bit_errors}")

    return {
        "pred_classes": pred_classes,
        "pred_one_hot": pred_one_hot,
        "true_one_hot": true_one_hot
    }
y_pred_svc = svc_clf.predict(X_test_scaled)

check_predictions_svc(y_pred_svc, y_test)


=== Podgląd pierwszych predykcji (SVC) ===
   y_true  y_pred  correct
0       3       3     True
1       0       0     True
2       0       0     True
3       2       3    False
4       1       1     True
5       3       3     True
6       1       0    False
7       3       3     True
8       1       1     True
9       0       0     True 

=== Przykładowe one-hot (prawda vs predykcja) ===
i=0: true=[0 0 0 1], pred=[0 0 0 1]
i=1: true=[1 0 0 0], pred=[1 0 0 0]
i=2: true=[1 0 0 0], pred=[1 0 0 0]
i=3: true=[0 0 1 0], pred=[0 0 0 1]
i=4: true=[0 1 0 0], pred=[0 1 0 0]

Liczba próbek: 300
Liczba błędnych próbek: 33
Accuracy: 0.8900
Suma błędów bitowych: 66


{'pred_classes': array([3, 0, 0, 3, 1, 3, 0, 3, 1, 0, 0, 1, 1, 1, 1, 2, 2, 0, 1, 1, 0, 0,
        2, 0, 2, 0, 3, 1, 3, 3, 1, 2, 0, 1, 0, 0, 2, 1, 0, 0, 1, 1, 1, 3,
        0, 2, 3, 3, 0, 2, 2, 3, 2, 1, 3, 0, 1, 1, 3, 3, 0, 3, 1, 0, 1, 3,
        1, 0, 2, 0, 1, 3, 2, 3, 1, 2, 3, 0, 1, 1, 3, 2, 2, 2, 2, 1, 2, 2,
        1, 2, 1, 0, 1, 0, 0, 1, 3, 0, 3, 0, 2, 2, 0, 0, 2, 3, 1, 1, 3, 2,
        0, 1, 1, 1, 3, 1, 2, 0, 3, 1, 2, 1, 3, 3, 2, 2, 1, 2, 1, 0, 2, 3,
        3, 2, 0, 0, 1, 2, 2, 3, 0, 3, 0, 1, 2, 1, 3, 1, 3, 3, 3, 2, 0, 1,
        0, 2, 1, 2, 2, 2, 1, 0, 0, 3, 0, 2, 0, 3, 2, 1, 0, 2, 2, 1, 3, 3,
        0, 0, 2, 1, 1, 2, 3, 2, 1, 2, 0, 3, 0, 1, 3, 3, 2, 1, 1, 3, 1, 2,
        1, 3, 2, 0, 2, 1, 3, 0, 0, 2, 1, 1, 2, 2, 3, 1, 3, 1, 2, 0, 3, 1,
        2, 3, 0, 1, 3, 0, 0, 0, 2, 3, 1, 0, 0, 3, 3, 2, 3, 3, 0, 2, 0, 3,
        3, 1, 2, 3, 0, 3, 0, 0, 0, 0, 0, 2, 3, 2, 3, 2, 1, 2, 2, 1, 0, 0,
        1, 3, 2, 3, 3, 3, 3, 1, 0, 0, 3, 1, 2, 1, 2, 1, 3, 0, 0, 1, 0, 3,
        2, 0, 2, 1, 2,

In [34]:
import plotly.express as px
from sklearn.metrics import confusion_matrix

# --- 1. Obliczamy macierz konfuzji dla testu ---
cm_svc = confusion_matrix(y_test, y_test_pred)

# --- 2. Rysunek ---
fig = px.imshow(
    cm_svc,
    text_auto=True,
    labels=dict(x="Predykcja", y="Rzeczywistość", color="Liczba"),
    x=[f"pred_{i}" for i in range(cm_svc.shape[0])],
    y=[f"true_{i}" for i in range(cm_svc.shape[0])],
    color_continuous_scale="Blues",
    title="Macierz konfuzji – SVC (RBF kernel)"
)

fig.update_layout(width=700, height=600)
fig.show()

